In [13]:
# Carga paquetes
import pandas as pd
import numpy as np
import warnings
from scipy.stats import skew

warnings.filterwarnings("ignore", category=UserWarning)  # ignora los warnings

In [18]:
# Configuración para evitar la notación científica
pd.options.display.float_format = '{:.0f}'.format

# Carga de la base
#inpath = "E:/OneDrive/Datasets/Tuboleta/Segmentacion/"
inpath = "C:/Users/diego.torres/OneDrive/Datasets/Tuboleta/Segmentacion/"

tb_ini = pd.read_csv(inpath + 'resultados_segmentacion_databricks.csv',delimiter='|',decimal=".")
tb_ini.fillna(0, inplace=True)

print('Dimensiones:')
print(tb_ini.shape)
print(tb_ini.columns)

Dimensiones:
(817004, 18)
Index(['Cliente', 'Eventos', 'Recaudo', 'Boletas', 'ATP', 'Valor', 'FIRSTNAME',
       'LASTNAME', 'ID_NUMBER', 'FECHA_DE_NACIMIENTO', 'EMAIL', 'Edad',
       'Rango_de_edad', 'Genero', 'CIUDAD', 'REGION', 'DEPARTAMENTO', 'PAIS'],
      dtype='object')


In [19]:
tb = tb_ini[['Cliente','Eventos','Recaudo','Boletas','ATP','Valor','EMAIL','Rango_de_edad']].\
    sort_values(by='Recaudo', ascending=False).\
    head(200)
#tb = tb_ini[(tb_ini['Eventos'] == 0) | (tb_ini['Recaudo'] == 0) | (tb_ini['Boletas'] == 0)]
print('Dimensiones:')
print(tb.shape)
print(tb.head(5))

Dimensiones:
(200, 8)
               Cliente  Eventos    Recaudo  Boletas    ATP     Valor  \
533400  10229230962591        1 1345832000     2724 494065      Alto   
93803   10228232382954      556 1131144261    72214  15664      Alto   
286881  10228564924593       30 1078635103    32110  33592  Muy Alto   
121398  10228342293221       89  857604112     7614 112635  Muy Alto   
136855  10228361265078      391  849024064    33818  25106      Alto   

                                   EMAIL Rango_de_edad  
533400             diegom@walterkolm.com      SIN DATO  
93803     gestor.planetario@tuboleta.com       18 a 24  
286881  esilva.cucutadeportivo@gmail.com      SIN DATO  
121398                tebankno@gmail.com       18 a 24  
136855          empresario1@tuboleta.com      SIN DATO  


In [20]:
# Escritura
tb.to_excel(inpath +'resultados_segmentacion_databricks.xlsx',
               index=False, # El argumento 'index=False' evita que se incluyan los índices en el archivo CSV
               #sep = '|',
               #decimal = '.'
               )